In [1]:
import pandas as pd
import numpy as np

In [2]:
# variables
batchNo = "2"
building = "CARPARK1"
month = "June"
year = "2022"
reportName = "HISTORY TRANSACTION"
fileFormat = "csv"
path = "batch " + batchNo + "/" + building + "/" + building + " " + month + " " + year  + " " + reportName + "." + fileFormat
path

'batch 2/CARPARK1/CARPARK1 June 2022 HISTORY TRANSACTION.csv'

In [3]:
df = pd.read_csv(path)
df["TIME"] = pd.to_datetime(df["TIME"], dayfirst=True)
df.sort_values("TIME", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5008 entries, 0 to 5007
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   S/NO.     5008 non-null   int64         
 1   TIME      5008 non-null   datetime64[ns]
 2   STATION   5008 non-null   object        
 3   IU        5008 non-null   object        
 4   CASHCARD  5008 non-null   object        
 5   VEHICLE   0 non-null      float64       
 6   CARDTYPE  5008 non-null   object        
 7   AMOUNT    5008 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 352.1+ KB


In [4]:
df

,S/NO.,TIME,STATION,IU,CASHCARD,VEHICLE,CARDTYPE,AMOUNT
0,0,2022-06-01 06:43:00,Entry,C13470461157,0,NaN,Season,$0.00
1,1,2022-06-01 07:23:00,Entry,C13471728958,0,NaN,,$0.00
2,2,2022-06-01 07:33:00,Entry,C13475420908,0,NaN,Season,$0.00
3,3,2022-06-01 07:49:00,Entry,C13471356018,0,NaN,,$0.00
4,4,2022-06-01 07:55:00,Entry,C13475456517,0,NaN,Hourly,$0.00
...,...,...,...,...,...,...,...,...
5003,5003,2022-06-30 23:47:00,Exit,C13471516941,1.00E+15,NaN,Hourly,$3.00
5004,5004,2022-06-30 23:53:00,Exit,C13473638414,1.11E+15,NaN,Hourly,$3.00
5005,5005,2022-06-30 23:57:00,Exit,C13475475757,1.11E+15,NaN,Hourly,$3.00
5006,5006,2022-06-30 23:58:00,Exit,C13475080974,1.11E+15,NaN,Hourly,$3.00


In [5]:
# remove the random spaces in STATION attribute
def strip(station):
    return station.strip()
df["STATION"] = df["STATION"].apply(strip)

In [6]:
# collect relevant data needed to match records in a list (using a list so i can iterate through easily)
data = []
for index, row in df.iterrows():
    id, time, station, iu, cashcard, amount = row["S/NO."], row["TIME"], row["STATION"], row["IU"], row["CASHCARD"], row["AMOUNT"]
    data.append({"id":id,
                "time":time, 
                "station":station,
                "iu":iu,
                "amount": amount,
                "cashcard": cashcard})

In [7]:
# match exits to entries
for i in range(len(data)):
    record = data[i]
    if record["station"] == "Entry":
        for j in range(i+1, len(data)):
            exit_record = data[j]
            if exit_record["station"] == "Exit" and exit_record["iu"] == record["iu"] and exit_record["time"] >= record["time"]:
                data[i]["exitTime"] = exit_record["time"]
                data[i]["amount"] =  exit_record["amount"]
                data[i]["cashcard"] = exit_record["cashcard"]
                break

# create a new dict to store the exit times by S/NO.
exitTimes = {}
for record in data:
    if record["station"] == "Entry":
        if "exitTime" not in record:
            record["exitTime"] = np.NaN
        exitTimes[record["id"]] = {"exitTime": record["exitTime"],
                                    "amount": record["amount"],
                                    "cashcard": record["cashcard"]}

In [8]:
# filter for Entry records
entry_df = df[df["STATION"] == "Entry"]

# use a list to order the exit times according to the sequence of records in entry_df (using S/NO. to identify the record)
exitTimesList = []
exitAmountList = []
exitCashcardList = []
for i in entry_df["S/NO."]:
    exitTimesList.append(exitTimes[i]["exitTime"])
    exitAmountList.append(exitTimes[i]["amount"])
    exitCashcardList.append(exitTimes[i]["cashcard"])

# add exit times to the dataset
entry_df["EXITDATETIME"] = exitTimesList
entry_df["AMOUNT"] = exitAmountList
entry_df["CASHCARD"] = exitCashcardList

# remove unnecessary column
entry_df.drop(columns=["STATION"], inplace=True)

# rename TIME column to ENTRYTIME
entry_df.rename(columns={"TIME":"ENTRYDATETIME"}, inplace=True)


/var/folders/t6/rlydfpt525n3gds3g246tfxw0000gn/T/ipykernel_15499/436772112.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry_df["EXITDATETIME"] = exitTimesList
/var/folders/t6/rlydfpt525n3gds3g246tfxw0000gn/T/ipykernel_15499/436772112.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry_df["AMOUNT"] = exitAmountList
/var/folders/t6/rlydfpt525n3gds3g246tfxw0000gn/T/ipykernel_15499/436772112.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [9]:
# double check if there is any errors in matching
entry_df[entry_df.EXITDATETIME < entry_df.ENTRYDATETIME]

,S/NO.,ENTRYDATETIME,IU,CASHCARD,VEHICLE,CARDTYPE,AMOUNT,EXITDATETIME


In [10]:
# # separte time out from the datetime rows
# def getTime(datetimeObject):
#     try:
#         return datetimeObject.time()
#     except ValueError:
#         return datetimeObject
    
    
# entry_df["ENTRYTIME"] = entry_df["ENTRYDATETIME"].apply(getTime)
# entry_df["EXITTIME"] = entry_df["EXITDATETIME"].apply(getTime)

In [11]:
# save to CSV
path =  "batch " + batchNo + "/outputFiles/entryExitMatched/" + building + "entryExitMatched.csv"
entry_df.to_csv(path, index=False)